In [7]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

In [9]:
path = 'attendance'
images= []
classNames = []
myList = os.listdir(path)
print(myList)

for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['ayushi.jpg', 'jaspreet.jpg', 'sahil.jpg']
['ayushi', 'jaspreet', 'sahil']


In [10]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

def markAttendance(name):
    with open('attendance.csv','r+')as f:
        myDataList = f.readlines()
        print(myDataList)
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M')
            f.writelines(f'\n{name},{dtString}')
        
encodeListKnown = findEncodings(images)
print('encoding complete, values found =' , len(encodeListKnown))

encoding complete, values found = 3


In [ ]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
 
    imgSmall = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    faceCurFrame= face_recognition.face_locations(imgSmall)
    encodeCurFrame = face_recognition.face_encodings(imgSmall,faceCurFrame)
    
    for encodeFace,faceLoc in zip(encodeCurFrame, faceCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        #print(faceDis)
        matchIndex = np.argmin(faceDis)
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            #print(name)
            y1,x2,y2,x1= faceLoc
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),1)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1)
            markAttendance(name)
            
    cv2.imshow('Webcam',img)
    cv2.waitKey(1)